In [ ]:
!pip install -U openmim
!mim install mmcv-full

In [ ]:
!git clone https://github.com/open-mmlab/mmsegmentation.git

In [ ]:
%cd /home/rongjian/yolov7-main/wandb
!git clone https://github.com/open-mmlab/mmcv.git
%cd mmcv
!pip install -r requirements.txt


In [ ]:
%cd /home/rongjian/yolov7-main/wandb/mmsegmentation
!pip install -v -e .

In [ ]:
%cd /home/rongjian/yolov7-main/wandb
%mkdir datas
%mkdir datas/ann
%mkdir datas/ann/training
%mkdir datas/ann/validation
%mkdir datas/images
%mkdir datas/images/training
%mkdir datas/images/validation

In [ ]:
import os
from shutil import copy
from sys import exit
import tqdm
path = "/home/rongjian/yolov7-main/wandb/dataset/semantic_drone_dataset/original_images" #文件夹目录
path_map = "/home/rongjian/yolov7-main/wandb/dataset/semantic_drone_dataset/label_images_semantic"
files= os.listdir(path) #得到文件夹下的所有文件名称

for i in range(len(files)):
    if i<360:
        copy(path+'/'+files[i], '/home/rongjian/yolov7-main/wandb/datas/images/training')
        copy(path_map+'/'+files[i][:-4]+'.png', '/home/rongjian/yolov7-main/wandb/datas/ann/training')
    elif 359<i<400:
        copy(path+'/'+files[i], '/home/rongjian/yolov7-main/wandb/datas/images/validation')
        copy(path_map+'/'+files[i][:-4]+'.png', '/home/rongjian/yolov7-main/wandb/datas/ann/validation')

In [ ]:
%cd /home/rongjian/yolov7-main/wandb/mmsegmentation

In [ ]:
%%writefile configs/_base_/models/danet_r50-d8.py
norm_cfg = dict(type='SyncBN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained='open-mmlab://resnet50_v1c',
    backbone=dict(
        type='ResNetV1c',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        dilations=(1, 1, 2, 4),
        strides=(1, 2, 1, 1),
        norm_cfg=norm_cfg,
        norm_eval=False,
        style='pytorch',
        contract_dilation=True),
    decode_head=dict(
        type='DAHead',
        in_channels=2048,
        in_index=3,
        channels=512,
        pam_channels=64,
        dropout_ratio=0.1,
        num_classes=23,
        norm_cfg=norm_cfg,
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
    auxiliary_head=dict(
        type='FCNHead',
        in_channels=1024,
        in_index=2,
        channels=256,
        num_convs=1,
        concat_input=False,
        dropout_ratio=0.1,
        num_classes=23,
        norm_cfg=norm_cfg,
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=0.4)),
    # model training and testing settings
    train_cfg=dict(),
    test_cfg=dict(mode='whole'))

In [ ]:
%%writefile mmseg/datasets/my_dataset.py
import mmcv
from mmcv.utils import print_log

from ..utils import get_root_logger
from .builder import DATASETS
from .custom import CustomDataset


@DATASETS.register_module()

class Mydataset(CustomDataset):
   
    CLASSES = ('unlabeled', 'paved-area', 'dirt', 'grass', 'gravel', 'water',
               'rocks', 'pool', 'vegetation', 'roof', 'wall', 'window', 'door', 'fence', 
               'fence-pole', 'person', 'dog', 'car', 'bicycle', 'tree', 'bald-tree', 'ar-marker', 
               'obstacle')

    PALETTE = [[0, 0, 0], [128, 64, 128], [130, 76, 0], [0, 102, 0], [112, 103, 87], [28, 42, 168], 
               [48, 41, 30], [0, 50, 89], [107, 142, 35], [70, 70, 70], [102, 102, 156], [254, 228, 12], 
               [254, 148, 12], [190, 153, 153], [153, 153, 153], [255, 22, 96], [102, 51, 0], [9, 143, 150], 
               [119, 11, 32], [51, 51, 0], [190, 250, 190], [112, 150, 146], [2, 135, 115]]


    def __init__(self, **kwargs):
        super(Mydataset, self).__init__(
            img_suffix='.jpg',
            seg_map_suffix='.png',
            ignore_index=255,
            **kwargs)
        assert self.file_client.exists(self.img_dir)

    def load_annotations(self,
                         img_dir,
                         img_suffix,
                         ann_dir,
                         seg_map_suffix=None,
                         split=None):
        

        img_infos = []
        if split is not None:
            with open(split) as f:
                for line in f:
                    name = line.strip()
                    img_info = dict(filename=name + img_suffix)
                    if ann_dir is not None:
                        ann_name = name 
                        seg_map = ann_name + seg_map_suffix
                        img_info['ann'] = dict(seg_map=seg_map)
                    img_infos.append(img_info)
        else:
            for img in mmcv.scandir(img_dir, img_suffix, recursive=True):
                img_info = dict(filename=img)
                if ann_dir is not None:
                    seg_img = img
                    seg_map = seg_img.replace(
                        img_suffix,  seg_map_suffix)
                    img_info['ann'] = dict(seg_map=seg_map)
                img_infos.append(img_info)

        print_log(f'Loaded {len(img_infos)} images', logger=get_root_logger())
        return img_infos

In [ ]:
%%writefile configs/_base_/datasets/my_dataset.py
dataset_type = 'Mydataset' #上一步中你定义的数据集的名字
data_root = '/home/rongjian/yolov7-main/wandb/datas' #数据集存储路径
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
crop_size = (768,768)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(768,768), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(768,768),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=5,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='images/training', #训练图像路径
        ann_dir='ann/training', #训练mask路径
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='images/validation',  #验证图像路径
        ann_dir='ann/validation',  #验证mask路径
        pipeline=test_pipeline),
    #我的数据集没有测试集，和验证集用同一路径，问题不大
    test=dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='images/validation',  #测试图像路径
        ann_dir='ann/validation',  #测试mask路径
        pipeline=test_pipeline))

In [ ]:
%%writefile mmseg/datasets/__init__.py
from .ade import ADE20KDataset
from .builder import DATASETS, PIPELINES, build_dataloader, build_dataset
from .chase_db1 import ChaseDB1Dataset
from .cityscapes import CityscapesDataset
from .coco_stuff import COCOStuffDataset
from .custom import CustomDataset
from .dark_zurich import DarkZurichDataset
from .dataset_wrappers import (ConcatDataset, MultiImageMixDataset,
                               RepeatDataset)
from .drive import DRIVEDataset
from .hrf import HRFDataset
from .isaid import iSAIDDataset
from .isprs import ISPRSDataset
from .loveda import LoveDADataset
from .night_driving import NightDrivingDataset
from .pascal_context import PascalContextDataset, PascalContextDataset59
from .potsdam import PotsdamDataset
from .stare import STAREDataset
from .voc import PascalVOCDataset
import sys
sys.path.append('..')
from .my_dataset import Mydataset
__all__ = [
    'CustomDataset', 'build_dataloader', 'ConcatDataset', 'RepeatDataset',
    'DATASETS', 'build_dataset', 'PIPELINES', 'CityscapesDataset',
    'PascalVOCDataset', 'ADE20KDataset', 'PascalContextDataset',
    'PascalContextDataset59', 'ChaseDB1Dataset', 'DRIVEDataset', 'HRFDataset',
    'STAREDataset', 'DarkZurichDataset', 'NightDrivingDataset',
    'COCOStuffDataset', 'LoveDADataset', 'MultiImageMixDataset',
    'iSAIDDataset', 'ISPRSDataset', 'PotsdamDataset','Mydataset'
]

In [ ]:
%%writefile configs/_base_/schedules/schedule_20k.py
optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0005)
optimizer_config = dict()
# learning policy
lr_config = dict(policy='poly', power=0.9, min_lr=1e-4, by_epoch=False)
runner = dict(type='EpochBasedRunner', max_epochs=100) #按epoch的方式进行迭代
checkpoint_config = dict(by_epoch=True, interval=50) #每多少次迭代保存一次模型
evaluation = dict(interval=5, metric='mIoU')  # 每多少次迭代计算一次指标


In [ ]:
%%writefile configs/pspnet/pspnet_r50-d8_512x512_160k_ade20k.py 
_base_ = [
    '../_base_/models/danet_r50-d8.py',
    '../_base_/datasets/my_dataset.py', '../_base_/default_runtime.py',
    '../_base_/schedules/schedule_20k.py'
]
model = dict(
    decode_head=dict(num_classes=23), auxiliary_head=dict(num_classes=23))

In [ ]:
!python tools/train.py configs/pspnet/pspnet_r50-d8_512x512_160k_ade20k.py  

In [ ]:
!python tools/test.py configs/pspnet/pspnet_r50-d8_512x512_160k_ade20k.py work_dirs/pspnet_r50-d8_512x512_160k_ade20k/latest.pth --opacity 1 --eval mIoU --show-dir out

In [ ]:
import cv2
import matplotlib.pyplot as plt
path = '/home/rongjian/yolov7-main/wandb/datas/images/validation'
files = os.listdir(path)
# 使用matplotlib展示多张图片
num = len(files)
j = 0
while j<len(files):
    
    for i in range(0,3):
        if i==2:
            img = cv2.imread('/home/rongjian/yolov7-main/wandb/mmsegmentation/out'+'/'+files[j][:-4]+'.jpg')
            title="predict"
        #行，列，索引
            plt.subplot(1,3,i+1)
        #plt.figure(figsize=(2, 2))
            plt.imshow(img)
            plt.title(title,fontsize=8)
            plt.xticks([])
            plt.yticks([])
    
        elif i==1:
            img = cv2.imread('/home/rongjian/yolov7-main/wandb/RGB_color_image_masks/RGB_color_image_masks'+'/'+files[j][:-4]+'.png')
            title="ground ture"
        #行，列，索引
            plt.subplot(1,3,i+1)
            #plt.figure(figsize=(2,2))
            plt.imshow(img)
            plt.title(title,fontsize=8)
            plt.xticks([])
            plt.yticks([])
        elif i==0:
            img = cv2.imread('/home/rongjian/yolov7-main/wandb/datas/images/validation'+'/'+files[j][:-4]+'.jpg')
            title="image"
        #行，列，索引
            plt.subplot(1,3,i+1)
            #plt.figure(figsize=(2, 2))
            plt.imshow(img)
            plt.title(title,fontsize=8)
            plt.xticks([])
            plt.yticks([])   
    plt.show()
    j = j+1
